# Imports

You will need to import MNE in order to use its functions to load and preprocess your data. Numpy is an extremely useful package used for mathematical and matrix operations. Pandas is the most convenient way of reading event codes from CSV files, but can also be used for analyzing tables of data.

If you wish to run artifact blocking, you will also need to import the run_ab function from artifact_blocking.py. Make sure you have a copy of artifact_blocking.py in the same folder as your own code to ensure that Python can find the function to import. If you try this and Python still cannot find artifact_blocking, use the alternative loading method shown below.

Finally, the line "%matplotlib qt" ensures that any time you try to plot your data, it will appear in a separate pop-up window and not just embed itself in the notebook.

In [ ]:
%matplotlib qt
import mne
import numpy as np
import pandas as pd
from artifact_blocking import run_ab

### Alternative loading method for artifact blocking

You can directly tell Python to look in a specific directory for artifact_blocking.py by using the function sys.path.append().

In [ ]:
import sys
sys.path.append('D://Documents/EEG_Workshop/')
from artifact_blocking import run_ab

# Loading Data

MNE provides functions for reading a wide variety of neuroimaging file types. Our lab primarily uses two EEG systems, a 129-channel EGI HydroCel GSN and a 128-channel BioSemi ActiveTwo system. Data from both of these systems can be easily loaded into MNE as shown below. Also included is the find_events() function, which extracts the trigger pulses from the recording and creates an N x 3 matrix of events that can later be used for epoching. The first column of the matrix lists each event's onset time (in samples) and the third column lists its event code.

If you only care about a subset of your channels, you can set a list of channels you want to keep using the pick_channels() function, which will drop all other channels. Alternatively, you can use the drop_channels() function to drop a specific set of channels and keep the rest.

If you need to re-code your events using external CSV files, you can use the code provided in the fifth cell below. This will overwrite the third column of the events matrix to contain the proper event codes, rather than the codes that appeared in the trigger channel during the recording.


## EGI

In [ ]:
# Set the path to the current session's EEG recording
filepath = 'D://Documents/EEG_Workshop/data/Adult_08_T.mff'

# Load the EEG file and extract trigger times
eeg = mne.io.read_raw_egi(filepath, preload=True)
eeg.rename_channels({'E129': 'Cz'})
montage = mne.channels.make_standard_montage('GSN-HydroCel-129')
eeg = eeg.set_montage(montage)
events = mne.find_events(eeg)

## BioSemi

In [ ]:
# Set the path to the current session's EEG recording
filepath = 'D://Documents/EEG_Workshop/data/CTpitchTapEEG_sub01_block1.bdf'
montage_file = 'D://Documents/EEG_Workshop/data/CTpitchTapEEG_sub01.sfp'

# Load the EEG file and extract trigger times
eeg = mne.io.read_raw_bdf(filepath, preload=True)
montage = mne.channels.read_custom_montage(montage_file)
eeg = eeg.set_montage(montage)
events = mne.find_events(eeg)

## Picking & Dropping Channels

In [ ]:
channels = ['E%s' % i for i in range(1, 129)]  # Example shorthand for keeping channels E1-E128
eeg = eeg.pick_channels(channels)

In [ ]:
channels = ['E1', 'E2', 'E3']  # Channels to drop
eeg = eeg.drop_channels(channels)

## Reading Event Codes

In [ ]:
# Select only events with a specific trigger code (e.g., 1)
# Only necessary if the number of trigger pulses in your recording does not match the number of events listed in your event code files
events = events[events[:, 2] == 1]

# Set the list of event code files for this session
evcode_files = ['D://Documents/EEG_Workshop/data/Vid_Trg_files/TriWav6min1.csv',
                'D://Documents/EEG_Workshop/data/Vid_Trg_files/TriWav6min2.csv',
                'D://Documents/EEG_Workshop/data/Vid_Trg_files/TriWav6min3.csv']

# Re-code events
evcodes = []
for f in evcode_files:
    evcodes.append(pd.read_csv(f, header=None))
evcodes = np.concatenate(evcodes).flatten()
events[:, 2] = evcodes

# Filtering

MNE allows you to apply filters to your data simply by calling the .filter() method on your EEG object. This function will work regardless of whether your data is formatted as epochs or continuous data, but note that it is generally preferable to filter your data before epoching in order to avoid edge artifacts. The .filter() method allows allows you to perform highpass, lowpass, or bandpass filtering using an FIR or IIR filter. The settings for both types of filters are explained below. **It is recommended that you filter your data prior to epoching.**

**To avoid confusion, note that MNE names its filter settings as if it were constructing a bandpass filter.** For example, you would run a 1 Hz highpass filter by setting l_freq = 1, not h_freq = 1. This is because the edge of the highpass filter forms the **lower** edge of the pass band. The edge of the lowpass filter forms the **higher** edge of the pass band.

## FIR Filter Settings:

The first two parameters allow you to specify the frequencies at which you wish to highpass and/or lowpass filter. Note that these values define the edge frequency of the filter, not the -6 dB cutoff frequency.

- **l_freq**: Specifies the band edge of the highpass filter (i.e. the lowest unattenuated frequency). Set to None if you do not wish to highpass filter.
- **h_freq**: Specifies the band edge of the lowpass filter (i.e. the highest unattenuated frequency). Set to None if you do not wish to lowpass filter.

The next two parameters allow you to specify the width of the highpass and lowpass filters' transition bands. The -6 dB cutoff frequency will be in the center of the transition band.

- **l_trans_bandwidth**: Sets the width of the highpass filter transition band. Can be set to 'auto' to automatically choose based on the highpass frequency.
- **h_trans_bandwidth**: Sets the width of the lowpass filter transition band. Can be set to 'auto' to automatically choose based on the lowpass frequency.

The next three parameters define the length and shape of the filter's window, as well as the phase of the filter.

- **filter_length**: Specifies the temporal length of the filter. Can be set to 'auto' to automatically choose based on the width of the transition bands, or you can manually specify by typing a string such as '10s' or '5000ms'.
- **fir_window**: Specifies the shape of the filter window. Set to 'hamming' for a Hamming window, 'hann' for a Hann/Hanning window, or 'blackman' for a Blackman window.
- **phase**: Set to 'zero' for a (non-causal) zero-phase filter. Set to 'zero-double' to perform the same (non-causal) zero-phase filter forwards and then backwards over the data. Set to 'minimum' for a (causal) minimum-phase filter.

The final parameter allows you to choose which channels you wish to apply your filter to.

- **picks**: Leave as None to filter all channels, or set to a list of channel names or indices to filter only those specific channels.

In [ ]:
# FIR Filtering
l_freq = .5
h_freq = 95
l_trans_bandwidth = 'auto'
h_trans_bandwidth = 'auto'
filter_length = 'auto'
fir_window = 'hamming'
phase = 'zero'
picks = None
eeg = eeg.filter(l_freq, h_freq, method='fir', fir_design='firwin', l_trans_bandwidth=l_trans_bandwidth, h_trans_bandwidth=h_trans_bandwidth, filter_length=filter_length, fir_window=fir_window, phase=phase, pad='reflect_limited', picks=picks)

## IIR (Butterworth) Filter Settings:

When running a Butterworth filter, you will only need to specify a highpass and/or lowpass filter cutoff frequency, the filter order, and which channels you wish to filter.

- **l_freq**: Specifies the -6 dB cutoff frequency of the highpass filter. Set to None if you do not wish to highpass filter.
- **h_freq**: Specifies the -6 dB cutoff frequency of the lowpass filter. Set to None if you do not wish to lowpass filter.
- **order**: Specifies the filter order.
- **picks**: Leave as None to filter all channels, or set to a list of channel names or indices to filter only those specific channels.

In [ ]:
# IIR Filtering
l_freq = .5
h_freq = 95
order = 4
picks = None
eeg = eeg.filter(l_freq, h_freq, method='iir', iir_params=dict(ftype='butter', order=order, output='sos'), picks=picks)

# Bad Channel Rejection & Interpolation

To manually mark electrodes as bad via visual inspection, simply plot your data with MNE's interactive data viewer and click on the names of any that look bad. Upon closing the data viewer, MNE will update your EEG object with a list of all the channels you marked as bad. You can view this list by looking at eeg.info['bads']. Various other MNE functions, including common average referencing and ICA, will automatically exclude channels marked as bad from their calculations. You can also use the interpolate_bads() function to interpolate all channels marked as bad using the spherical spline method. Both bad channel marking and interpolation can be performed on continuous or epoched data.

### Settings:

- **reset_bads**: If True, interpolated channels will no longer be considered bad channels by other MNE functions. If False, interpolated channels will still be treated as bad channels by other MNE functions.

In [ ]:
# Use MNE's interactive data viewer to mark bad channels
eeg.plot()

In [ ]:
n_interp = len(eeg.info['bads'])  # Save the number of bad channels as a variable in case we need it for ICA
eeg = eeg.interpolate_bads(reset_bads=True)  # Interpolate all bad channels

# Artifact Blocking

Artifact blocking is a method developed by our lab, and is particularly useful for cleaning the irregular artifacts that occur in infant data. Note that you will need to import the run_ab function from the file artifact_blocking.py in order to use artifact blocking, as it uses house-built code rather than being part of MNE. **Artifact blocking must be run on continuous data; it cannot be applied to epoched data.** Note that the EEG object will be modified in-place.

### Settings:

- **threshold**: Set the artifact detection threshold (in microvolts).
- **method**: Set to 'window' to run artifact blocking with a sliding window. Set to 'total' to run artifact blocking on the entire session at once (requires more memory).

In [ ]:
# Artifact Blocking
run_ab(eeg, threshold=75, method='window')

# Referencing

MNE provides a variety of data referencing schema, but the most common methods in scalp EEG are common average referencing and unipolar referencing. For information on how to apply a bipolar referencing scheme, see https://mne.tools/stable/generated/mne.set_bipolar_reference.html. The set_eeg_reference() function can be used on both continuous and epoched data.

### Settings:

- **ref_channels**: Set to 'average' to apply common average referencing. Set to the name of a single channel to use that channel as the reference. Set to a list of channel names to use the average of those electrodes as the reference. *Note that average referencing will automatically exclude any electrodes currently marked as bad.*

In [ ]:
# Referencing
eeg = eeg.set_eeg_reference(ref_channels='average')

# Epoching

In order to epoch your data, you just need to use the matrix that was generated from your event triggers when you loaded your data, and to define the time period that each epoch should inlude relative to the event onsets. Note that MNE will not allow any epochs to overlap, and will return an error if any of your time windows intersect with one another.

### Settings

- **tmin**: Defines how many seconds after the event onset each epoch should begin. Set to a negative value if you want the epoch to begin prior to the event onset.
- **tmax**: Defines how many seconds after the event onset each epoch should end. Set to a negative value if you want the epoch to end prior to the event onset.

In [ ]:
# Epoch data around the trigger pulses in the EEG recording
eeg = mne.Epochs(eeg, events, tmin=-.5, tmax=1., preload=True)

**Note:** To access all epochs from only one condition, you can use the event code or condition name as an index, like so:

In [ ]:
eeg['4']

# Baseline Correction
In order to baseline correct each epoch by its own baseline period (e.g. 200 ms prior to event onset), MNE provides the apply_baseline() function. Baseline correction is specific to epoched data, and can not be used on continuous data.

### Settings:
- **start**: The start time in seconds of the window to be used as a baseline, relative to the event onset (*not* relative to the start of the epoch). For example, set to -0.2 to start your baseline period 200 ms prior to each event. If None, baseline period will begin at the start of the epoch.
- **stop**: The end time in seconds of the window to be used as a baseline, relative to the event onset (*not* relative to the start of the epoch). For instance, set to 0 to end your baseline at the onset of each event. If None, baseline period will extend until the end of the epoch.

In [ ]:
# Baseline correct
start = None
stop = 0
eeg = eeg.apply_baseline((start, stop))

# Epoch Rejection

The drop_bad() function can be used to reject epochs whose peak-to-peak amplitudes exceed a given threshold on one or more channels. Rejected epochs will be directly removed from your data object. This function is specific to epoched data.

### Settings:

- **threshold**: The maximum acceptible peak-to-peak amplitude (in microvolts).

In [ ]:
# Reject epochs using peak-to-peak amplitude
threshold = 150
eeg = eeg.drop_bad({'eeg': threshold / 1000000.})

# Downsampling

Downsampling can be useful for reducing the memory load of your data processing and speed up runtimes. MNE's resample() function uses a frequency-domain filter to low-pass filter and downsample the data. You can also use this function to upsample your data, though this is not common practice.

**Note**: You should downsample your data only after epoching. If you downsample the raw data and then try to create epochs using downsampled triggers, you will introduce jitter into your event onset times.

## Settings:
- **sfreq**: The sampling rate you to which your data will be downsampled.
- **pad**: The type of padding you wish to use for the filter. Set to "edge" to pad each epoch with its own first/last voltage value. Set to "reflect-limited" to pad each epoch with a mirrored version of itself.

In [ ]:
eeg = eeg.resample(sfreq=100., pad='edge')

# ICA Artifact Rejection

Independent Component Analysis (ICA) is a popular method for separating muscle artifacts and other sources of noise from actual brain activity. MNE performs ICA in a two-stage process. First, it decomposes the channel data into its principal components via PCA in order to decorrelate the channels and re-scale them to unit variance. It then runs ICA on the PCA components. If *n_components* is set less than the number of channels in the data, dimensionality reduction is performed by dropping the last *n_channels* - *n_components* PCs prior to running ICA.

ICA can be run either on continuous or epoched data. However, if using continuous data, be aware that highly irregular activity during break periods may reduce the quality of your ICA solution. ICA also has high memory requirements, so running it on epoched and/or downsampled data may be necessary if using a system with limited RAM.

**Note**: ICA is sensitive to baseline drift, and requires the data be high-pass filtered at (at least) 0.5 Hz or 1 Hz prior to running ICA.

### How many components to use?

If you have not altered your data in any way prior to running ICA, the maximum number of components you can decompose will be identical to the number of channels in your dataset. However, certain procedures such as re-referencing and interpolation can reduce the rank of your data, consequently reducing the maximum number of components you can actually decompose. Common average referencing will reduce the rank of your data by one. Channel interpolation further reduces the rank of your data by one for each interpolated channel. This is because the interpolated version of a channel is simply a weighted sum of other channels, meaning that its variance is entirely accounted for by those other channels. **You should therefore select a number of components equal to the number of channels in your data, subtracting one if you used an average reference, and subtracting an additional one for each channel you have interpolated.**

### Settings:
- **n_components**: The number of ICA components that will be estimated. Cannot be greater than *max_pca_components*. See above for notes on best practices for choosing an appropriate number of components.
- **method**: Specifies which ICA algorithm to use. Options are "infomax" (Infomax), "fastica" (FastICA), and "picard" (PICARD: Preconditioned ICA for Real Data).
- **random_state**: ICA is non-deterministic, but you can set a specific random seed (any integer) to guarantee the same result when re-running ICA on the same data. If left as None, the random seed will be selected at random and results will differ between runs.

For information on more advanced ICA usage in MNE, see https://mne.tools/stable/auto_tutorials/preprocessing/plot_40_artifact_correction_ica.html#sphx-glr-auto-tutorials-preprocessing-plot-40-artifact-correction-ica-py.

In [ ]:
# Run ICA
n_components = len(eeg.ch_names) - (1 + n_interp)
method = 'infomax'
random_state = None
ica = mne.preprocessing.ICA(n_components=n_components, max_pca_components=n_components, random_state=random_state, method=method)
ica.fit(eeg)

### Identifying Artifactual Components
Once you have finished running ICA, MNE offers several plotting functions for visualizing your components and manually marking those which appear to be artifactual. Any components you mark as artifactual will be recorded in the ICA object once you close the plotting window.

In [ ]:
# View the topographies of all components to manually identify bad components (Click on component's name to mark as bad)
ica.plot_components()

In [ ]:
# View time courses of all components to manually identify bad components (Click on component's time course to mark as bad)
ica.plot_sources(eeg)

In [ ]:
# View detailed information on specific components
picks = [0, 1, 2, 3]
ica.plot_properties(eeg, picks=picks)

### Removing bad components

Subsequently running ica.apply(eeg) will remove from your data any components that you have marked as bad.

In [ ]:
# Remove bad components from data
ica.apply(eeg)

### Saving and Loading ICA Solutions
If you wish to save your ICA solution to a file and load it back up later, you can use the functions below to do so. Note that MNE requires all ICA savefiles to end in "-ica.fif".

In [ ]:
# Save ICA solution to a file
filename = 'C://Users/jpazd/Downloads/Adult_12_D-ica.fif'
ica.save(filename)

In [ ]:
# Load ICA solution from a file
filename = 'C://Users/jpazd/Downloads/Adult_12_D-ica.fif'
ica = mne.preprocessing.read_ica(filename)

# Surface Laplacian / Current Source Density Transform

It is common to perform a surface Laplacian transform prior to connectivity analyses. MNE provides the compute_current_source_density() function to transform your data using a surface Laplacian. *Note that this technique was added to MNE in late 2019 and requires v0.20 or higher.*

### Settings:

- **sphere**: Defines the model of the head as a sphere of the form (x, y, z, r), where x, y, and z are the coordinates of the center of the sphere and r is the radius of the sphere in meters. If you have added a digitization of the electrode locations to your EEG object, you can also set sphere to 'auto' to use that.
- **lambda2**: Sets the regularization parameter (MNE default is 1e-05).
- **stiffness**: Sets the stiffness of the spherical spline (MNE default is 4).
- **n_legendre_terms**: Sets the number of Legendre terms to evaluate (MNE default is 50).

In [ ]:
eeg = mne.preprocessing.compute_current_source_density(eeg, 
                                                       sphere=(0, 0, 0, .1),
                                                       lambda2=1e-05, 
                                                       stiffness=4, 
                                                       n_legendre_terms=50, 
                                                       copy=False)

# Creating Regions of Interest

Sometimes you may want to analyze entire regions of interest rather than individual channels. The code below will help you group your data into regions of interest. It first creates new data channels by averaging the electrodes within each region you define. It then replaces your original channel x time data with an ROI x time structure (if running before epoching) or your event x channel x time data with an even x ROI x time structure (if running after epoching).

To use this code, you should only need to change the ROI definitions at the top.

In [ ]:
# Define regions of interest -- this should be a dictionary where ROI names are mapped to lists of the channels they contain
rois = dict(
    Fz=['E4', 'E10', 'E11', 'E16', 'E18', 'E19'],
    Cz=['E7', 'E31', 'E55', 'E80', 'E106'],
    Pz=['E61', 'E62', 'E67', 'E72', 'E77', 'E78']
)

# Build a weight matrix that maps channels to regions of interest
proj_vec = np.zeros((len(rois), len(eeg.ch_names)), dtype=float)
for i, roi in enumerate(rois):
    rois[roi] = mne.pick_channels(eeg.ch_names, include=rois[roi])
    proj_vec[i, rois[roi]] = 1
    proj_vec[i, :] /= proj_vec[i, :].sum()

# Replace old EEG object with a new one organized by ROI
roi_info = mne.create_info(ch_names=[k for k in rois], sfreq=eeg.info['sfreq'], ch_types='eeg')
if isinstance(eeg, mne.io.BaseRaw):
    eeg = mne.io.RawArray(np.dot(proj_vec, eeg.get_data()), roi_info)
elif isinstance(eeg, mne.BaseEpochs):
    eeg = mne.EpochsArray(np.array([np.dot(proj_vec, epoch) for epoch in eeg.get_data()]), roi_info, events=eeg.events, tmin=eeg.tmin)
else:
    raise ValueError('Can only convert channels to ROIs within Raw and Epochs objects, not %s.' % type(eeg))

# Saving & Loading Preprocessed Epochs

If you wish to save your preprocessed epochs to a file, so that you do not need to re-process them the next time you look at your data, you can simply call the save() method of an EEG object. To load the data from that file later, simply use the mne.read_epochs() function. Note that MNE requires all savefiles for epoched data to end with "-epo.fif".

In [ ]:
# Save epochs to file
eeg.save('C://Users/jpazd/Downloads/Adult_12_D-epo.fif')

In [ ]:
# Load epochs from file
eeg = mne.read_epochs('C://Users/jpazd/Downloads/Adult_12_D-epo.fif', preload=True)